In [1]:
# for each ligand, generate 5 protein-ligand descriptors
# 1 protein is the matching protein, 4 are random proteins
# descriptors are saved as a .npy file

# also outputs a csv file, containing
# lig_id, pro_id, descriptor path, score
# score = 1 if lig_id & pro_id matches, 0 otherwise

In [29]:
from htmd.ui import *
from htmd.molecule.voxeldescriptors import *
from pathlib import Path
import numpy as np
import pickle as pk
from multiprocessing import Pool
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
#Supress default INFO logging

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [31]:
np.random.seed = 1
num_random_proteins = 4

In [32]:
ligand_df = pd.read_csv('../data/csv/ligand_data.csv', index_col=False)

In [33]:
# split ligands here
train_size = int(len(ligand_df) * 0.90)

train_df = ligand_df.iloc[:train_size]
test_df = ligand_df.iloc[train_size:]

In [34]:
def gen_data(df, base_dest):
    Path(base_dest).mkdir(exist_ok=True, parents=True)
    
    all_lig_idxs = []
    all_protein_idxs = []
    dests = []
    score = []

    def gen_pro_lig_voxel_pairs(lig_idx, protein_idxs, path):
        with open(path, 'rb') as f:
            lig_features, centers = pk.load(f)

        for pidx in protein_idxs:
            mol = Molecule('../data/filtered_data/'+str(pidx).zfill(4)+'_pro_cg.pdb', keepaltloc='all')
            
            pro_features, _ = getVoxelDescriptors(mol, usercenters=np.array(centers), voxelsize=1, method='CUDA')
            pro_features = pro_features.reshape(24, 24, 24, pro_features.shape[1])
            pro_features = pro_features[:,:,:,0]
            combined = np.concatenate((pro_features[:,:,:,None], lig_features[:,:,:,None]), axis=3)
            combined = combined.astype(np.float32)

            dest = base_dest + '/' + str(pidx).zfill(4) + '_pro_' + str(lig_idx).zfill(4) + '_lig.npy'
            np.save(dest, combined)

            all_lig_idxs.append(lig_idx)
            all_protein_idxs.append(pidx)
            dests.append(dest)
            score.append(1 if pidx == lig_idx else 0)
            
    for row in df.itertuples():
        df_idx = row[0]
        lig_idx = row[1]
        path = row[2]
        
        # generate 4 random proteins to pair
        r = range(0,len(df)-1)
        selection = np.random.choice(r, num_random_proteins, replace=False)
        selection = [len(df)-1 if s == df_idx else s for s in selection]
        # convert them into protein idxs
        protein_idxs = list(df.iloc[selection].id.values)
        protein_idxs.append(lig_idx)
        
        gen_pro_lig_voxel_pairs(lig_idx, protein_idxs, path)
        
    pro_lig_record = pd.DataFrame({'lig_id': all_lig_idxs, 'pro_id': all_protein_idxs, 'dests': dests, 'score': score}, index=None)
    return pro_lig_record

In [35]:
train_dest = '../processed_data/pro_lig_voxels/train'
test_dest = '../processed_data/pro_lig_voxels/test'

In [36]:
import multiprocessing
from functools import partial

def parallelize_dataframe(df, func):
    num_cores = multiprocessing.cpu_count()-1  #leave one free to not freeze machine
    num_partitions = num_cores #number of partitions to split dataframe
    df_split = np.array_split(df, num_partitions)
    pool = multiprocessing.Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [38]:
#gen_data(train_df, train_dest, 'train_lig_pro_pairs.csv')
#train_lig_pro_paris = parallelize_dataframe(train_df, partial(gen_data, base_dest=train_dest))
#gen_data(test_df, test_dest, '../data/csvs/test_lig_pro_pairs.csv')

In [39]:
test_lig_pro_pairs = parallelize_dataframe(test_df, partial(gen_data, base_dest=test_dest))
test_lig_pro_pairs.to_csv('test_lig_pro_pairs.csv')

/home/alex/miniconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
/home/alex/miniconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
/home/alex/miniconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
/home/alex/miniconda3/lib/python3.6/site-packages/htmd/molecule/reader

In [40]:
train_lig_pro_pairs = parallelize_dataframe(train_df, partial(gen_data, base_dest=train_dest))
train_lig_pro_pairs.to_csv('train_lig_pro_pairs.csv')

/home/alex/miniconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
/home/alex/miniconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
/home/alex/miniconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
/home/alex/miniconda3/lib/python3.6/site-packages/htmd/molecule/reader

In [28]:
train_size

2700